In [1]:
import sys  
sys.path.insert(0, '../src')
import config
import util

import data_processing as dp
import numpy as np

import random
import os
import torch
from torch.utils.data import Dataset

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

import torch.nn as nn
import torch.optim as optim
import time
from sklearn.metrics import accuracy_score, roc_auc_score

from PIL import Image

In [2]:
random.seed(config.SEED)
np.random.seed(config.SEED)
torch.manual_seed(config.SEED)
os.environ["PYTHONHASHSEED"] = str(config.SEED)
torch.multiprocessing.freeze_support()

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

Device: cuda


In [4]:
disease_filter_list=['Atelectasis']
position_filter_list=['PA']

In [5]:
df_data, labels   = dp.load_data()
labels

['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Effusion',
 'Emphysema',
 'Fibrosis',
 'Hernia',
 'Infiltration',
 'Mass',
 'Nodule',
 'Pleural_Thickening',
 'Pneumonia',
 'Pneumothorax']

In [6]:
if disease_filter_list:
    labels = disease_filter_list
    #df_data = df_data[df_data['Finding Labels'].isin(disease_filter_list)]
          
if position_filter_list:
    df_data = df_data[df_data['View Position'].isin(position_filter_list)]

In [7]:
df_data.head(5)

,Patient ID,Patient Age,Patient Gender,Follow Up,View Position,Image Index,Image_path,Finding Labels
0,1,58,M,0,PA,00000001_000.png,..\data\raw\images\00000001_000.png,Cardiomegaly
1,1,58,M,1,PA,00000001_001.png,..\data\raw\images\00000001_001.png,Cardiomegaly|Emphysema
2,1,58,M,2,PA,00000001_002.png,..\data\raw\images\00000001_002.png,Cardiomegaly|Effusion
3,2,81,M,0,PA,00000002_000.png,..\data\raw\images\00000002_000.png,No Finding
4,3,81,F,0,PA,00000003_000.png,..\data\raw\images\00000003_000.png,Hernia


In [8]:
labels

['Atelectasis']

In [9]:
df_data, dict_labels = dp.multi_hot_label(df_data, labels)
df_train, df_test    = dp.make_train_test_split(df_data)

In [10]:
dict_labels

{0: 'Atelectasis'}

In [11]:
df_train, df_val   = dp.train_test_split(df_train) 
df_train, df_test, df_val = df_train.reset_index(drop=True), df_test.reset_index(drop=True), df_val.reset_index(drop=True)
len(df_train)

44971

In [12]:
len(df_test)

11096

In [13]:
len(df_val)

11243

In [16]:
class NihDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        
        self.dataframe = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        image  = Image.open(self.dataframe.loc[idx, 'Image_path']).convert('L')  # via .getband(), some images are know to have 4 channels. Here we convert them to 1-channel grayscale.
        target = self.dataframe.loc[idx, 'Labels']
        
        if self.transform:
            image = self.transform(image)
        
        return image, target

In [17]:
def load_data(dataframe, transform=None, batch_size=32, shuffle=True, num_workers=4):
    '''
    Data Loader with batch loading and transform.
    '''
    image_data = NihDataset(dataframe, transform=transform)
    pin = device=='cpu'
    num_workers=num_workers*int(device!='cpu')
    loader = torch.utils.data.DataLoader(image_data, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=pin)
    return loader

In [18]:
class SimpleNet(nn.Module):
    def __init__(self, labels):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5)  # stride=1, padding=0
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(3, 8, 5)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(8, 16, 5)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 10 * 10, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)  # 14 classes
        
        self.out = nn.Linear(64, len(labels))
        
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))  # size: (batch_size*)3channels*110*110
        x = self.pool2(F.relu(self.conv2(x)))  # size: (batch_size*)8channels*53*53
        x = self.pool3(F.relu(self.conv3(x)))  # size: (batch_size*)8channels*53*53
        x = x.view(-1, 16 * 10 * 10)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        out = torch.sigmoid(self.out(x))
        #x = F.softmax(self.fc3(x))                # change to softmax if multiclass
        return out
    
class Net(nn.Module):
    def __init__(self, labels):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5)  # stride=1, padding=0
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(3, 8, 5)
        self.fc1 = nn.Linear(8 * 25 * 25, 60)
        self.fc2 = nn.Linear(60, 20)
        self.fc3 = nn.Linear(20, len(labels))  # 2 classes
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # size: (batch_size*)3channels*110*110
        x = self.pool(F.relu(self.conv2(x)))  # size: (batch_size*)8channels*53*53
        x = x.view(-1, 8 * 25 * 25)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = torch.sigmoid(self.fc3(x))  # change to softmax if multiclass
        return x

In [19]:
def multi_category_loss_fn(outputs, targets):
    
    tl = []
    for o,t in zip(outputs.T, targets.T):
        tl.append(nn.BCELoss()(o, t))
        
    return sum(tl) / len(tl)

In [32]:
model = SimpleNet(dict_labels).to(device)
#model = Net(dict_labels).to(device)
len(dict_labels)

1

In [33]:
criterion = nn.BCEWithLogitsLoss()  # change to CrossEntropyLoss if  multiclass
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [34]:
num_epochs=config.NUM_EPOCHS
num_epochs=3
num_epochs

3

In [35]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop((112,112)),
        transforms.RandomHorizontalFlip(),  # data augmentation
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((128,128)),
        transforms.CenterCrop(112),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [36]:
train_data_loader = load_data(df_train, transform=data_transforms['train'], shuffle=False, batch_size=config.TRAIN_BATCH_SIZE, num_workers=0)
val_data_loader   = load_data(df_val  , transform=data_transforms['test'] , shuffle=False, batch_size=config.VAL_BATCH_SIZE  , num_workers=0)

In [37]:
train_losses, val_losses = [], []

In [38]:
model.train()

SimpleNet(
  (conv1): Conv2d(1, 3, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [39]:
running_loss = 0.0
epoch = 1

In [40]:
for i, data in enumerate(train_data_loader, 0):
    torch.cuda.empty_cache()
    
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.type(torch.FloatTensor).to(device)
            
    # forward + backward + optimize
    outputs = model(inputs)
    loss = multi_category_loss_fn(outputs, labels)
    
    # zero the parameter gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # print statistics
    running_loss += loss.item()
    print(".", end ="")
    if i % 10 == 9:    # print every 10 mini-batches
        print(f'\nEpoch: {epoch + 1:>2} , Bacth: {i + 1:>3} , loss: {running_loss / (i+1)}')
        #print(f"Average time per batch: {(time.time()-start_time)/(i+1)} secs")
        #running_loss = 0.0

.

In [41]:
val_loss = 0

In [42]:
with torch.no_grad():
    model.eval()
    Y_prob, Y_pred, Y_true = [], [], []
    for idx, (images, labels) in enumerate(val_data_loader):
        probs = model(images.to(device))
        labels = labels.type(torch.FloatTensor).to(device)
        val_loss += multi_category_loss_fn(probs, labels)
        predicted = probs > 0.5
                
        probs = probs.cpu().detach().numpy().astype(float)
        predicted = predicted.cpu().detach().numpy().astype(float)
        labels = labels.cpu().detach().numpy().astype(float)
        Y_prob.append(probs)
        Y_pred.append(predicted)
        Y_true.append(labels)
        
        

In [43]:
Y_prob = np.concatenate(Y_prob, axis=0)
Y_pred = np.concatenate(Y_pred, axis=0)
Y_true = np.concatenate(Y_true, axis=0)
print()
        
train_losses.append(running_loss/len(train_data_loader))
val_losses.append(val_loss/len(val_data_loader))
acc = accuracy_score(Y_true, Y_pred)
roc = roc_auc_score(Y_true, Y_prob)
        
print(f"Epoch              : {epoch+1}/{num_epochs}")
print(f"Training Loss      : {train_losses[-1]}")
print(f"Validation Loss    : {val_losses[-1]}")
print(f"Validation Accuracy: {acc}")
print(f"Validation ROC     : {roc}")
            


Epoch              : 2/3
Training Loss      : 0.7397649884223938
Validation Loss    : 0.29458072781562805
Validation Accuracy: 0.92
Validation ROC     : 0.3383152173913043
